In [ ]:
import requests
from bs4 import BeautifulSoup
import json

headers = {"User-Agent": "Mozilla/5.0"}
url = "https://www.bbc.com/news"

resp = requests.get(url, headers=headers)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

# Try static headlines first
headlines = [h.get_text(strip=True) for h in soup.select("h3.gs-c-promo-heading__title")]

if headlines:
    print("Headlines from BBC News front page:")
    for i, title in enumerate(headlines[:10], 1):
        print(f"{i}. {title}")
else:
    # Fallback: parse embedded JSON on individual article pages
    script = soup.select_one("#__NEXT_DATA__")
    if script:
        data = json.loads(script.text)
        page = next(v for k,v in data["props"]["pageProps"]["page"].items() if k.startswith("@"))
        headline = next(c for c in page["contents"] if c["type"] == "headline")
        print("Headline:", headline["model"]["blocks"][0]["model"]["text"])
    else:
        print("No headlines found with either method.")